In [5]:
import os
import json
import numpy as np
from collections import defaultdict
from scipy.spatial.transform import Rotation as R

# 디렉토리 설정
base_dirs = [
    "/home/zed_box/Documents/1_ArUco_cap",
    "/home/zed_box/Documents/2_ArUco_cap",
    "/home/zed_box/Documents/3_ArUco_cap"
]
output_dir = "/home/zed_box/Documents/Correct_ArUco"
os.makedirs(output_dir, exist_ok=True)

# 파일 이름에서 뷰와 카메라 추출
def parse_filename(filename):
    parts = filename.split('_')
    view = parts[0]
    cam = parts[2]
    return view, cam

# 데이터 구조: data[view][cam][marker_id] = list of marker dicts
data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# 파일 수집 및 데이터 적재
for base_dir in base_dirs:
    for fname in os.listdir(base_dir):
        if not fname.endswith('.json'):
            continue
        view, cam = parse_filename(fname)
        with open(os.path.join(base_dir, fname), 'r') as f:
            content = json.load(f)
            for marker_id, marker_data in content.items():
                data[view][cam][marker_id].append(marker_data)

# 회전 평균
def average_quaternion(quaternions):
    r = R.from_quat(quaternions)
    return R.mean(r).as_quat()

# 위치 평균
def average_position(positions):
    return np.mean(positions, axis=0)

# 마커 정리 및 저장
for view in data:
    for cam in data[view]:
        corrected = {}
        for marker_id, entries in data[view][cam].items():
            if len(entries) < 2:
                continue  # 최소 두 개 이상 있어야 평균

            positions = [np.array([m['position_m']['x'], m['position_m']['y'], m['position_m']['z']]) for m in entries]
            quaternions = [np.array([m['rotation_quat']['x'], m['rotation_quat']['y'],
                                     m['rotation_quat']['z'], m['rotation_quat']['w']]) for m in entries]

            avg_pos = average_position(positions)
            avg_quat = average_quaternion(quaternions)

            corrected[marker_id] = {
                "position_m": {"x": float(avg_pos[0]), "y": float(avg_pos[1]), "z": float(avg_pos[2])},
                "rotation_quat": {"x": float(avg_quat[0]), "y": float(avg_quat[1]),
                                  "z": float(avg_quat[2]), "w": float(avg_quat[3])},
                "corners_pixel": entries[0]["corners_pixel"]
            }

        output_path = os.path.join(output_dir, f"{view}_{cam}_corrected.json")
        with open(output_path, 'w') as f:
            json.dump(corrected, f, indent=4)

print("보정된 결과가 다음 디렉토리에 저장되었습니다:", output_dir)


보정된 결과가 다음 디렉토리에 저장되었습니다: /home/zed_box/Documents/Correct_ArUco


In [4]:
import os
import json
import numpy as np
from collections import defaultdict
from scipy.spatial.transform import Rotation as R

# ----------------------------
# 디렉토리 설정 및 그룹 정의
# ----------------------------
base_dirs = [
    "/home/zed_box/Documents/1_ArUco_cap",
    "/home/zed_box/Documents/2_ArUco_cap",
    "/home/zed_box/Documents/3_ArUco_cap"
]
output_dir = "/home/zed_box/Documents/Correct_ArUco"
os.makedirs(output_dir, exist_ok=True)

# 동일한 x 그룹
x_groups = [
    ["1", "2", "4"],
    ["3", "7"],
    ["5", "6", "8"]
]

# 동일한 y 그룹
y_groups = [
    ["1", "6"],
    ["2", "5"],
    ["4", "5", "8"]
]

# 파일 이름에서 뷰와 카메라 추출
def parse_filename(filename):
    parts = filename.split('_')
    view = parts[0]
    cam = parts[2]
    return view, cam

# 데이터 구조: data[view][cam][marker_id] = list of marker dicts
data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# 파일 수집 및 데이터 적재
for base_dir in base_dirs:
    for fname in os.listdir(base_dir):
        if not fname.endswith('.json'):
            continue
        view, cam = parse_filename(fname)
        with open(os.path.join(base_dir, fname), 'r') as f:
            content = json.load(f)
            for marker_id, marker_data in content.items():
                data[view][cam][marker_id].append(marker_data)

# ----------------------------
# 보정 함수 정의
# ----------------------------
def average_quaternion(quaternions):
    quaternions = np.array(quaternions)
    ref_q = quaternions[0]
    for i in range(1, len(quaternions)):
        if np.dot(ref_q, quaternions[i]) < 0:
            quaternions[i] = -quaternions[i]
    mean_q = np.mean(quaternions, axis=0)
    return mean_q / np.linalg.norm(mean_q)

def average_position(positions):
    return np.mean(positions, axis=0)

def average_group_value(data, group, axis):
    values = []
    for marker_id in group:
        if marker_id in data:
            values.append(data[marker_id]["position_m"][axis])
    if values:
        return round(sum(values) / len(values), 6)
    return None

# ----------------------------
# 뷰/카메라 별 보정 및 저장
# ----------------------------
for view in data:
    for cam in data[view]:
        corrected_data = {}

        for marker_id, entries in data[view][cam].items():
            if len(entries) < 2:
                continue
            positions = [np.array([m['position_m']['x'], m['position_m']['y'], m['position_m']['z']]) for m in entries]
            quaternions = [np.array([m['rotation_quat']['x'], m['rotation_quat']['y'],
                                     m['rotation_quat']['z'], m['rotation_quat']['w']]) for m in entries]

            avg_pos = average_position(positions)
            avg_quat = average_quaternion(quaternions)

            corrected_data[marker_id] = {
                "position_m": {"x": float(avg_pos[0]), "y": float(avg_pos[1]), "z": float(avg_pos[2])},
                "rotation_quat": {"x": float(avg_quat[0]), "y": float(avg_quat[1]),
                                  "z": float(avg_quat[2]), "w": float(avg_quat[3])},
                "corners_pixel": entries[0]["corners_pixel"]
            }

        for group in x_groups:
            avg_x = average_group_value(corrected_data, group, "x")
            if avg_x is not None:
                for marker_id in group:
                    if marker_id in corrected_data:
                        corrected_data[marker_id]["position_m"]["x"] = avg_x

        for group in y_groups:
            avg_y = average_group_value(corrected_data, group, "y")
            if avg_y is not None:
                for marker_id in group:
                    if marker_id in corrected_data:
                        corrected_data[marker_id]["position_m"]["y"] = avg_y

        output_path = os.path.join(output_dir, f"{view}_{cam}_corrected.json")
        with open(output_path, 'w') as f:
            json.dump(corrected_data, f, indent=4)

print("[INFO] 총 8개의 보정 결과가 저장되었습니다:", output_dir)


[INFO] 총 8개의 보정 결과가 저장되었습니다: /home/zed_box/Documents/Correct_ArUco
